# DATA ANALYSIS MODULE

In [3]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from itertools import product

# Define the country code at the beginning of the notebook
country = 'EA20'  # Change to 'GR', 'DE' OR 'EA20' as needed
lower_country = country.lower()


# EDA

## ***DATA VISUALIZATION COMPONENT***

##### Timelines

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# Read the column.md file and store it in a dictionary
column_info = {}
with open(f'columns_{lower_country}.md', 'r', encoding='utf-8') as file:
    for line in file:
        line = line.strip()
        if line:
            col, desc = line.split(': ', 1)  # Use maxsplit=1 to avoid splitting issues
            column_info[col] = {'feature': col, 'description': desc}

# Read the data from the CSV file
data = pd.read_csv(f'INFUSED_DATA/merged_data_{country}.csv')

# Convert the TIME_PERIOD column to datetime
data['TIME_PERIOD'] = pd.to_datetime(data['TIME_PERIOD'])

# Define the number of plots per set and filter only columns that exist in the data
plots_per_set = 6
filtered_column_info = {k: v for k, v in column_info.items() if k in data.columns}
total_features = len(filtered_column_info)

# Create the plots
for i in range(0, total_features, plots_per_set):
    fig, axes = plt.subplots(nrows=3, ncols=2, figsize=(15, 15))
    axes = axes.flatten()
    
    for j, (feature, info) in enumerate(list(filtered_column_info.items())[i:i+plots_per_set]):
        ax = axes[j]
        ax.plot(data['TIME_PERIOD'], data[feature])
        ax.set_title(info['description'])
        ax.set_xlabel('TIME_PERIOD')
        ax.set_ylabel(info['feature'])
    
    # Remove empty subplots if there are less than 6 features in this set
    for k in range(j+1, plots_per_set):
        fig.delaxes(axes[k])
    
    plt.tight_layout()
    plt.show()


##### Distribution diagrams

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Read the column.md file and store it in a dictionary
column_info = {}
with open(f'columns_{lower_country}.md', 'r', encoding='utf-8') as file:
    for line in file:
        line = line.strip()
        if line:
            col, desc = line.split(': ', 1)  # Use maxsplit=1 to avoid splitting issues
            column_info[col] = {'feature': col, 'description': desc}

# Read the data from the CSV file
data = pd.read_csv(f'INFUSED_DATA/merged_data_{country}.csv')

# Convert the TIME_PERIOD column to datetime
data['TIME_PERIOD'] = pd.to_datetime(data['TIME_PERIOD'])

# Define the number of plots per set and filter only columns that exist in the data
plots_per_set = 6
filtered_column_info = {k: v for k, v in column_info.items() if k in data.columns}
total_features = len(filtered_column_info)

# Create the plots
for i in range(0, total_features, plots_per_set):
    fig, axes = plt.subplots(nrows=3, ncols=2, figsize=(15, 15))
    axes = axes.flatten()
    
    for j, (feature, info) in enumerate(list(filtered_column_info.items())[i:i+plots_per_set]):
        ax = axes[j]
        sns.histplot(data[feature], kde=True, ax=ax)  # Use seaborn to create histogram with density line
        ax.set_title(info['description'])
        ax.set_xlabel(feature)
        ax.set_ylabel('Frequency')
    
    # Remove empty subplots if there are less than 6 features in this set
    for k in range(j+1, plots_per_set):
        fig.delaxes(axes[k])
    
    plt.tight_layout()
    plt.show()


##### Boxplots

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Read the column.md file and store it in a dictionary
column_info = {}
with open(f'columns_{lower_country}.md', 'r', encoding='utf-8') as file:
    for line in file:
        line = line.strip()
        if line:
            col, desc = line.split(': ', 1)  # Use maxsplit=1 to avoid splitting issues
            column_info[col] = {'feature': col, 'description': desc}

# Read the data from the CSV file
data = pd.read_csv(f'INFUSED_DATA/merged_data_{country}.csv')

# Convert the TIME_PERIOD column to datetime
data['TIME_PERIOD'] = pd.to_datetime(data['TIME_PERIOD'])

# Function to calculate outliers percentage
def calculate_outliers(data, feature, threshold=1.5):
    q1 = data[feature].quantile(0.25)
    q3 = data[feature].quantile(0.75)
    iqr = q3 - q1
    lower_bound = q1 - threshold * iqr
    upper_bound = q3 + threshold * iqr

    outliers = data[(data[feature] < lower_bound) | (data[feature] > upper_bound)]
    outliers_count = outliers.shape[0]
    total_count = data.shape[0]
    outliers_percentage = (outliers_count / total_count) * 100
    
    return outliers_count, outliers_percentage

# Function to plot outliers
def plot_outliers(data, features, title):
    plots_per_page = 6
    rows, cols = 3, 2

    for i in range(0, len(features), plots_per_page):
        fig, axs = plt.subplots(rows, cols, figsize=(15, 15))
        axs = axs.flatten()

        for j, feature in enumerate(features[i:i+plots_per_page]):
            if feature in data.columns:  # Check if the feature exists in the data
                ax = axs[j]
                sns.boxplot(x=data[feature], ax=ax)
                outliers_count, outliers_percentage = calculate_outliers(data, feature)
                ax.set_title(f'Boxplot for {feature}\nOutliers: {outliers_count} ({outliers_percentage:.2f}%)')
                ax.set_xlabel(feature)

        # Remove any unused subplots
        for k in range(j+1, len(axs)):
            fig.delaxes(axs[k])

        plt.suptitle(title, fontsize=16)
        plt.tight_layout(rect=[0, 0, 1, 0.96])  # Adjust layout to make space for the main title
        plt.show()
        plt.close(fig)  # Close the figure to free memory

# List of features for outliers analysis
all_features = list(column_info.keys())

# Filter features to include only those present in the data
filtered_features = [feature for feature in all_features if feature in data.columns]

# Plot outliers
plot_outliers(data, filtered_features, 'Outliers Detection for All Features')


##### Data Transformation

In [5]:
import pandas as pd
import numpy as np

# Read the data from the updated CSV file
data = pd.read_csv(f'INFUSED_DATA/merged_data_{country}.csv')

# List of trend and VideoCount variables to convert to binary
variables = [
    'Russia_Ukraine_War_Trend', 
    'Israel_Gaza_War_Trend', 
    'Financial_Crisis_Trend', 
    'Climate_Change_Trend', 
    'Covid19_Trend',
    'Russia_Ukraine_War_VideoCount', 
    'Israel_Gaza_War_VideoCount', 
    'Financial_Crisis_VideoCount', 
    'Climate_Change_VideoCount', 
    'Covid19_VideoCount'
]

# Function to convert to binary based on mean
def convert_to_binary(df, col):
    threshold = df[col].mean()
    binary_col = f'{col}_Binary'
    df[binary_col] = np.where(df[col] >= threshold, 1, 0)

# Apply the function to each variable
for var in variables:
    convert_to_binary(data, var)

# Save the updated data to a new CSV file
data.to_csv(f'EDA_ML_DATA/{country}/updated_merged_data_{country}.csv', index=False)


##### Correlation Analysis

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Read the data from the updated CSV file
data = pd.read_csv(f'EDA_ML_DATA/{country}/updated_merged_data_{country}.csv')

# Convert the TIME_PERIOD column to datetime
data['TIME_PERIOD'] = pd.to_datetime(data['TIME_PERIOD'])

# Compute and plot correlation matrix for original data
original_data_no_time_period = data.drop(columns=['TIME_PERIOD'])
correlation_matrix_original = original_data_no_time_period.corr()
plt.figure(figsize=(20, 16))
sns.heatmap(correlation_matrix_original, annot=True, cmap='coolwarm', linewidths=.5)
plt.title('Heatmap of Feature Correlations')
plt.show()

# Extracting strong correlations and saving to CSV
def extract_strong_correlations(corr_matrix, threshold=0.60):
    corr_pairs = []
    for i in range(len(corr_matrix.columns)):
        for j in range(i+1, len(corr_matrix.columns)):
            if abs(corr_matrix.iloc[i, j]) >= threshold:
                corr_pairs.append((corr_matrix.columns[i], corr_matrix.columns[j], corr_matrix.iloc[i, j]))
    return corr_pairs

# Extract strong correlations
strong_correlations = extract_strong_correlations(correlation_matrix_original, threshold=0.60)

# Create a DataFrame for strong correlations
strong_correlations_df = pd.DataFrame(strong_correlations, columns=['Feature 1', 'Feature 2', 'Correlation'])

# Split the DataFrame into positive and negative correlations
positive_corr_df = strong_correlations_df[strong_correlations_df['Correlation'] > 0].copy()
negative_corr_df = strong_correlations_df[strong_correlations_df['Correlation'] < 0].copy()

# Sort by absolute value of correlation
positive_corr_df = positive_corr_df.reindex(positive_corr_df['Correlation'].abs().sort_values(ascending=False).index)
negative_corr_df = negative_corr_df.reindex(negative_corr_df['Correlation'].abs().sort_values(ascending=False).index)

# Concatenate the positive and negative correlations
sorted_strong_correlations_df = pd.concat([positive_corr_df, negative_corr_df])

# Save the strong correlations to a CSV file
sorted_strong_correlations_df.to_csv(f'EDA_ML_DATA/{country}/strong_correlations.csv', index=False)


##### Scatter Plots

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd

# Load the strong correlations DataFrame
strong_corr_features = pd.read_csv(f'EDA_ML_DATA/{country}/strong_correlations.csv')

# Define the exclusion keywords
exclude_keywords = ['BS', 'CISS', 'EU', 'ESTR', 'UN', 'ECB', 'Close', 'Volume', 'Deaths', 'Cases', 'Trend', 'VideoCount', 'Binary']

# Function to check if both features contain the same keyword
def contains_same_keyword(feature1, feature2, keywords):
    for keyword in keywords:
        if keyword in feature1 and keyword in feature2:
            return True
    return False

# Filter out rows where both 'Feature 1' and 'Feature 2' contain the same keyword
strong_corr_features = strong_corr_features[
    ~strong_corr_features.apply(lambda row: contains_same_keyword(row['Feature 1'], row['Feature 2'], exclude_keywords), axis=1)
]

# Define strong correlations (correlation > 0.75 or correlation < -0.75)
strong_corr_features = strong_corr_features[(strong_corr_features['Correlation'] > 0.75) | (strong_corr_features['Correlation'] < -0.75)]

# Split the features into positive and negative correlations
positive_corr_features = strong_corr_features[strong_corr_features['Correlation'] >= 0.75].sort_values(by='Correlation', ascending=False)
negative_corr_features = strong_corr_features[strong_corr_features['Correlation'] <= -0.75].sort_values(by='Correlation')

# Function to plot scatter plots with regression lines
def plot_correlations(corr_features, title_prefix, data):
    plots_per_page = 6
    rows, cols = 3, 2

    for i in range(0, len(corr_features), plots_per_page):
        fig, axs = plt.subplots(rows, cols, figsize=(15, 15))
        axs = axs.flatten()

        for j, (index, row) in enumerate(corr_features.iloc[i:i+plots_per_page].iterrows()):
            ax = axs[j]
            sns.scatterplot(data=data, x=row['Feature 1'], y=row['Feature 2'], ax=ax)
            sns.regplot(data=data, x=row['Feature 1'], y=row['Feature 2'], scatter=False, color='red', ax=ax)
            ax.set_title(f'{title_prefix}: {row["Feature 1"]} vs {row["Feature 2"]} (Correlation: {row["Correlation"]:.2f})')
            ax.set_xlabel(row['Feature 1'])
            ax.set_ylabel(row['Feature 2'])

        # Remove any unused subplots
        for j in range(len(corr_features.iloc[i:i+plots_per_page]), len(axs)):
            fig.delaxes(axs[j])

        plt.tight_layout()
        plt.show()
        plt.close(fig)  # Close the figure to free memory

# Load the data
data = pd.read_csv(f'EDA_ML_DATA/{country}/updated_merged_data_{country}.csv')

# Plot positive correlations
plot_correlations(positive_corr_features, 'Positive Correlation', data)

# Plot negative correlations
plot_correlations(negative_corr_features, 'Negative Correlation', data)


##### Groupwise Correlation Analysis

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from itertools import combinations

# Load the data
data = pd.read_csv(f'EDA_ML_DATA/{country}/updated_merged_data_{country}.csv')

# Define feature groups
economic_features = [
    f'GDP_GROWTH_{country}', f'INFLATION_{country}_VALUE', f'GOVDEBT_{country}',
    'ECB_EXCHANGE_RATES_USD_EUR', 'ECB_EXCHANGE_RATES_CNY_EUR',
    'ESTR_EU000A2QQF08_CI', 'ESTR_EU000A2X2A25_NT', 'ESTR_EU000A2X2A25_TT',
    f'CISS_{country}_SS_CIN', 'CISS_EA20_SS_BM', 'CISS_EA20_SS_FI',
    'CISS_EA20_SS_FX', 'CISS_EA20_SS_MM',
    f'BS_{country}_CSMCI', f'BS_{country}_FS_NY', f'BS_{country}_GES_NY', f'BS_{country}_MP_NY',
    f'BS_{country}_PT_NY', f'BS_{country}_SV_NY', f'UN_{country}_T_TOT', f'UN_{country}_M_TOT', 
    f'UN_{country}_F_TOT', f'UN_{country}_GT25_TOT', f'UN_{country}_LE25_TOT', 'Oil_Close_Price',
    'Oil_Volume', 'Gas_Close_Price', 'Gas_Volume', 'Corn_Close_Price', 'Corn_Volume',
    'Wheat_Close_Price', 'Wheat_Volume'
]
covid_features = [col for col in data.columns if 'Covid_' in col]
trend_features = [col for col in data.columns if col.endswith('Trend') or col.endswith('Trend_Binary')]
videocount_features = [col for col in data.columns if col.endswith('VideoCount') or col.endswith('VideoCount_Binary')]

# Function to create heatmap for a set of features
def plot_heatmap(features, title):
    subset = data[features].dropna()
    correlation_matrix = subset.corr()
    plt.figure(figsize=(20, 10))  # Adjust the figure size as needed
    sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', linewidths=.5)
    plt.title(f'Heatmap of {title}')
    plt.show()

# Plot heatmap for economic features separately
plot_heatmap(economic_features, 'Economic Features')

# Plot heatmap for covid features separately
plot_heatmap(covid_features, 'Covid Features')

# Plot heatmap for trend features separately
plot_heatmap(trend_features, 'Trend Features')

# Plot heatmap for videocount features separately
plot_heatmap(videocount_features, 'VideoCount Features')

# Function to create heatmap of correlations between two feature groups
def plot_intergroup_heatmap(group1, group2, title):
    subset = data[group1 + group2].dropna()
    correlation_matrix = subset.corr().loc[group1, group2]
    plt.figure(figsize=(20, 10))  # Adjust the figure size as needed
    sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', linewidths=.5)
    plt.title(f'Heatmap of {title}')
    plt.show()

# Define feature groups
groups = {
    'Economic Features': economic_features,
    'Covid Features': covid_features,
    'Trend Features': trend_features,
    'VideoCount Features': videocount_features
}

# Create heatmaps for correlations between different feature groups
group_pairs = list(combinations(groups.items(), 2))

for (group1_name, group1_features), (group2_name, group2_features) in group_pairs:
    plot_intergroup_heatmap(group1_features, group2_features, f'{group1_name} vs {group2_name}')


## ***PREPARATION AND SELECTION OF VARIABLE GROUPS FOR THE MACHINE LEARNING STAGE***

In [ ]:
import pandas as pd
from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.tools.tools import add_constant
import ast

# Function to calculate VIF
def calculate_vif(df, features):
    X = df[features]

    # Ensure columns are numeric
    for col in features:
        if not pd.api.types.is_numeric_dtype(X[col]):
            raise ValueError(f"The column {col} is not numeric.")

    X = X.dropna()  # Remove rows with NaN values
    X = add_constant(X)

    vif_data = pd.DataFrame()
    vif_data["Feature"] = X.columns
    vif_data["VIF"] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]
    return vif_data

# Read the CSV file with correlations
file_path = f'EDA_ML_DATA/{country}/strong_correlations.csv'
df = pd.read_csv(file_path)

# List of all unique variables from 'Feature 1' column
unique_variables = df['Feature 1'].unique()

# Create all possible groups of dependent-independent variables
groups = []

for dependent_var in unique_variables:
    # Select rows where 'Feature 1' is the dependent variable and the correlation is above 0.60
    relevant_rows = df[(df['Feature 1'] == dependent_var) & (df['Correlation'].abs() > 0.60)]

    # Get independent variables
    independent_vars = relevant_rows['Feature 2'].tolist()

    if independent_vars:
        groups.append((dependent_var, independent_vars))

# Convert groups to DataFrame
groups_df = pd.DataFrame(groups, columns=['Dependent Variable', 'Independent Variables'])

# Save the result to a CSV file
output_file_path = f'EDA_ML_DATA/{country}/dependent_independent_groups.csv'
groups_df.to_csv(output_file_path, index=False)

print(f'The dependent-independent groups have been saved to {output_file_path}')

# Read the CSV file with the groups
groups_file_path = f'EDA_ML_DATA/{country}/dependent_independent_groups.csv'
groups_df = pd.read_csv(groups_file_path)

# Read the CSV file with correlations
correlations_file_path = f'EDA_ML_DATA/{country}/strong_correlations.csv'
correlations_df = pd.read_csv(correlations_file_path)

# Read the dataset with actual values
dataset_file_path = f'EDA_ML_DATA/{country}/updated_merged_data_{country}.csv'
full_df = pd.read_csv(dataset_file_path)

# Process to check the independent variables of each group
results = []

for _, row in groups_df.iterrows():
    dependent_var = row['Dependent Variable']
    independent_vars = ast.literal_eval(row['Independent Variables'])  # Convert string list to list

    # Check correlations
    correlated_pairs = []
    for i, var1 in enumerate(independent_vars):
        for var2 in independent_vars[i+1:]:
            corr_row = correlations_df[
                ((correlations_df['Feature 1'] == var1) & (correlations_df['Feature 2'] == var2)) |
                ((correlations_df['Feature 1'] == var2) & (correlations_df['Feature 2'] == var1))
            ]
            if not corr_row.empty and abs(corr_row['Correlation'].values[0]) > 0.75:
                correlated_pairs.append((var1, var2, corr_row['Correlation'].values[0]))

    # Calculate VIF using the actual values from full_df
    try:
        vif_result = calculate_vif(full_df, independent_vars)
    except Exception as e:
        vif_result = pd.DataFrame({'Feature': independent_vars, 'VIF': [str(e)]*len(independent_vars)})

    # Save results
    results.append({
        'Dependent Variable': dependent_var,
        'Independent Variables': independent_vars,
        'Correlated Pairs': correlated_pairs,
        'VIF Result': vif_result
    })

# Create final DataFrame for all results
final_results = []

for result in results:
    dependent_var = result['Dependent Variable']
    for i, row in result['VIF Result'].iterrows():
        feature = row['Feature']
        vif = row['VIF']
        correlated_with = [pair for pair in result['Correlated Pairs'] if feature in pair[:2]]
        final_results.append({
            'Dependent Variable': dependent_var,
            'Feature': feature,
            'VIF': vif,
            'Correlated With': correlated_with
        })

final_results_df = pd.DataFrame(final_results)

# Save the final file
final_output_file = f'EDA_ML_DATA/{country}/final_vif_correlations_results.csv'
final_results_df.to_csv(final_output_file, index=False)

print(f"The final results have been saved to {final_output_file}")


##### VIF-High Correlation Testing

In [ ]:
import pandas as pd
from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.tools.tools import add_constant

# Function to calculate VIF
def calculate_vif(df, features):
    X = df[features]

    # Ensure columns are numeric
    for col in features:
        if not pd.api.types.is_numeric_dtype(X[col]):
            raise ValueError(f"The column {col} is not numeric.")

    X = X.dropna()  # Remove rows with NaN values
    X = add_constant(X)

    vif_data = pd.DataFrame()
    vif_data["Feature"] = X.columns
    vif_data["VIF"] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]
    return vif_data

# Define dependent and independent variables for Team-1
#dependent_var = f'UN_{country}_T_TOT'  # Replace with your dependent variable
#independent_vars = ['Covid19_Trend_Binary', 'ECB_EXCHANGE_RATES_USD_EUR', f'Covid_{country}_Cumulative_Deaths', 'Oil_Close_Price', 'Corn_Close_Price']  # Replace with your independent variables

# Define dependent and independent variables for Team-2
dependent_var = f'GOVDEBT_{country}'  # Replace with your dependent variable Israel_Gaza_War_VideoCount_Binary
independent_vars = ['BS_EA20_MP_NY', 'UN_EA20_T_TOT', f'Covid_{country}_Cumulative_Deaths', 'Oil_Close_Price']   # Replace with your independent variables

# Define dependent and independent variables for Team-3
#dependent_var = f'INFLATION_{country}_VALUE'  # Replace with your dependent variable
#independent_vars = ['Russia_Ukraine_War_VideoCount_Binary', 'Oil_Close_Price', f'Covid_{country}_Cumulative_Deaths', 'ECB_EXCHANGE_RATES_CNY_EUR', f'BS_{country}_FS_NY', f'UN_{country}_T_TOT', 'Covid19_Trend_Binary', 'ECB_EXCHANGE_RATES_USD_EUR']  # Replace with your independent variables

# Read the dataset with actual values
dataset_file_path = f'EDA_ML_DATA/{country}/updated_merged_data_{country}.csv'
full_df = pd.read_csv(dataset_file_path)

# Check correlations between independent variables
correlations_df = pd.read_csv(f'EDA_ML_DATA/{country}/strong_correlations.csv')

correlated_pairs = []
for i, var1 in enumerate(independent_vars):
    for var2 in independent_vars[i+1:]:
        corr_row = correlations_df[
            ((correlations_df['Feature 1'] == var1) & (correlations_df['Feature 2'] == var2)) |
            ((correlations_df['Feature 1'] == var2) & (correlations_df['Feature 2'] == var1))
        ]
        if not corr_row.empty and abs(corr_row['Correlation'].values[0]) > 0.75:
            correlated_pairs.append((var1, var2, corr_row['Correlation'].values[0]))

# Calculate VIF using the actual values from full_df
try:
    vif_result = calculate_vif(full_df, independent_vars)
except Exception as e:
    vif_result = pd.DataFrame({'Feature': independent_vars, 'VIF': [str(e)]*len(independent_vars)})

# Save results
results = [{
    'Dependent Variable': dependent_var,
    'Independent Variables': independent_vars,
    'Correlated Pairs': correlated_pairs,
    'VIF Result': vif_result
}]

# Create final DataFrame for the results
final_results = []

for result in results:
    dependent_var = result['Dependent Variable']
    for i, row in result['VIF Result'].iterrows():
        feature = row['Feature']
        vif = row['VIF']
        correlated_with = [pair for pair in result['Correlated Pairs'] if feature in pair[:2]]
        final_results.append({
            'Dependent Variable': dependent_var,
            'Feature': feature,
            'VIF': vif,
            'Correlated With': correlated_with
        })

final_results_df = pd.DataFrame(final_results)

# Save the final file
final_output_file = f'EDA_ML_DATA/{country}/testing_vif_correlations_results.csv'
final_results_df.to_csv(final_output_file, index=False)

print(f"The testing results have been saved to {final_output_file}")


# HYPOTHESIS TESTING

## ***Linear Regression - Random Forests - Gradient Boosting***

In [ ]:
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error
import numpy as np
import statsmodels.api as sm
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.metrics import make_scorer

# Read the dataset with actual values
dataset_file_path = f'EDA_ML_DATA/{country}/updated_merged_data_{country}.csv'
df = pd.read_csv(dataset_file_path)

# Convert TIME_PERIOD column to datetime
df['TIME_PERIOD'] = pd.to_datetime(df['TIME_PERIOD'])

# Define dependent and independent variables for Team-1  
dependent_var = f'UN_{country}_T_TOT'  # Replace with your dependent variable 
independent_vars = ['Covid19_Trend_Binary', 'ECB_EXCHANGE_RATES_USD_EUR', 'Oil_Close_Price', f'Covid_{country}_Cumulative_Deaths',  'Corn_Close_Price']  # Replace with your independent variables

# Define dependent and independent variables for Team-2
#  
#dependent_var = f'GOVDEBT_{country}'  # Replace with your dependent variable
#independent_vars = ['BS_EA20_MP_NY', f'Covid_{country}_Cumulative_Deaths', 'UN_EA20_T_TOT', 'Oil_Close_Price']   # Replace with your independent variables

# Create X and y
X = df[independent_vars]
y = df[dependent_var]

# Split the data into training and testing sets using random order
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Z-score normalization for all models
scaler_X = StandardScaler()
X_train_scaled = scaler_X.fit_transform(X_train)
X_test_scaled = scaler_X.transform(X_test)

scaler_y = StandardScaler()
y_train_scaled = scaler_y.fit_transform(y_train.values.reshape(-1, 1))
y_test_scaled = scaler_y.transform(y_test.values.reshape(-1, 1))

# Linear Regression Model
model_lr = LinearRegression()
model_lr.fit(X_train_scaled, y_train_scaled)

# Fit the model with statsmodels for detailed analysis
X_train_scaled_sm = sm.add_constant(X_train_scaled)  # Add constant term
model_sm = sm.OLS(y_train_scaled, X_train_scaled_sm).fit()

# Print coefficients, p-values, and check for significance
print("Linear Regression Coefficients and p-values:")
coefficients = model_sm.params
p_values = model_sm.pvalues
for coef, p_val, var in zip(coefficients, p_values, ['const'] + independent_vars):
    significance = "Significant" if p_val < 0.05 else "Not Significant"
    print(f"{var}: Coefficient = {coef:.4f}, p-value = {p_val:.4f} ({significance})")

# Make predictions with Linear Regression
predictions_scaled_lr = model_lr.predict(X_test_scaled)
predictions_lr = scaler_y.inverse_transform(predictions_scaled_lr)

# Calculate evaluation metrics for Linear Regression
r_squared_lr = model_lr.score(X_test_scaled, y_test_scaled)
mae_lr = mean_absolute_error(y_test, predictions_lr)
mse_lr = mean_squared_error(y_test, predictions_lr)
rmse_lr = np.sqrt(mse_lr)

# Calculate mean of actual values
mean_y_test = np.mean(y_test)

# Calculate percentage metrics
mae_lr_percent = (mae_lr / mean_y_test) * 100
rmse_lr_percent = (rmse_lr / mean_y_test) * 100
mse_lr_percent = (mse_lr / (mean_y_test ** 2)) * 100

print(f"Linear Regression - R-squared: {r_squared_lr}")
print(f"Linear Regression - Mean Absolute Error (MAE): {mae_lr} ({mae_lr_percent:.2f}%)")
print(f"Linear Regression - Mean Squared Error (MSE): {mse_lr} ({mse_lr_percent:.2f}%)")
print(f"Linear Regression - Root Mean Squared Error (RMSE): {rmse_lr} ({rmse_lr_percent:.2f}%)")
mean_y_test = np.mean(y_test)
print(f"Mean of dependent variable (y_test): {mean_y_test:.4f}")

# Create a dataframe to hold the actual and predicted values
results_df = df[['TIME_PERIOD', dependent_var]].copy()
results_df['Predicted_LR'] = np.nan
results_df.loc[X_test.index, 'Predicted_LR'] = predictions_lr

# Plot the actual values and predicted values over time for Linear Regression
plt.figure(figsize=(10, 6))
plt.plot(results_df['TIME_PERIOD'], results_df[dependent_var], label='Actual Values', color='b', linewidth = 0.8)
plt.plot(results_df['TIME_PERIOD'], results_df['Predicted_LR'], label='Predicted Values', color='r', linestyle = '--', marker = 'o', markersize = 1)
plt.xlabel('Time')
plt.ylabel(dependent_var)
plt.title(f'Actual vs Predicted Values for {dependent_var} - Linear Regression')
plt.legend()
plt.grid(True)
plt.show()

# Plotting the importance of variables based on p-values
plt.figure(figsize=(10, 6))
p_values_dict = {var: p_val for var, p_val in zip(['const'] + independent_vars, p_values)}
sorted_p_values = dict(sorted(p_values_dict.items(), key=lambda item: item[1]))

bars = plt.bar(sorted_p_values.keys(), -np.log10(list(sorted_p_values.values())), color=['gray' if var == 'const' else 'blue' for var in sorted_p_values.keys()])
plt.axhline(y=-np.log10(0.05), color='red', linestyle='--', label='Significance Level (0.05)')
plt.xlabel('Variables')
plt.ylabel('-log10(p-value)')
plt.title('Variable Significance Based on Hypothesis Testing')
plt.legend()
plt.grid(True)
plt.xticks(rotation=45)
plt.show()

# Random Forest Model using scaled data
print("\nRandom Forest Model Results:")
rf_model = RandomForestRegressor(n_estimators=100, random_state=42)
rf_model.fit(X_train_scaled, y_train_scaled.ravel())
rf_predictions_scaled = rf_model.predict(X_test_scaled)
rf_predictions = scaler_y.inverse_transform(rf_predictions_scaled.reshape(-1, 1))

# Calculate evaluation metrics for Random Forest
rf_mae = mean_absolute_error(y_test, rf_predictions)
rf_mse = mean_squared_error(y_test, rf_predictions)
rf_rmse = np.sqrt(rf_mse)

# Calculate percentage metrics for Random Forest
rf_mae_percent = (rf_mae / mean_y_test) * 100
rf_rmse_percent = (rf_rmse / mean_y_test) * 100
rf_mse_percent = (rf_mse / (mean_y_test ** 2)) * 100

print(f"Random Forest - Mean Absolute Error (MAE): {rf_mae} ({rf_mae_percent:.2f}%)")
print(f"Random Forest - Mean Squared Error (MSE): {rf_mse} ({rf_mse_percent:.2f}%)")
print(f"Random Forest - Root Mean Squared Error (RMSE): {rf_rmse} ({rf_rmse_percent:.2f}%)")

# Add RF predictions to the results dataframe
results_df['Predicted_RF'] = np.nan
results_df.loc[X_test.index, 'Predicted_RF'] = rf_predictions

plt.figure(figsize=(10, 6))
plt.plot(results_df['TIME_PERIOD'], y, label='Actual Values', color='b', linewidth = 0.8)
plt.plot(results_df['TIME_PERIOD'], results_df['Predicted_RF'], label='RF Predicted Values', color = 'r', linestyle = '--', marker = 'o', markersize = 1)
plt.xlabel('Time')
plt.ylabel(dependent_var)
plt.title(f'Actual vs Random Forest Predicted Values for {dependent_var}')
plt.legend()
plt.grid(True)
plt.show()

importances_rf = rf_model.feature_importances_
indices_rf = np.argsort(importances_rf)[::-1]
features_rf = [independent_vars[i] for i in indices_rf]

# Print feature importances for Random Forest
print("\nFeature Importances based on Gini Index (Random Forest):")
for feature_name, importance in zip(features_rf, importances_rf[indices_rf]):
    print(f"{feature_name}: {importance:.4f}")

plt.figure(figsize=(10, 6))
plt.bar(features_rf, importances_rf[indices_rf], color='blue')
plt.axhline(y=0.05, color='red', linestyle='--', label='Significance Level (0.05)')
plt.xlabel('Variables')
plt.ylabel('Importance')
plt.title('Variable Importance Based on Random Forest')
plt.xticks(rotation=45)
plt.legend()  # Add legend to show the significance level
plt.grid(True)
plt.show()

# Gradient Boosting Model using scaled data
print("\nGradient Boosting Model Results:")
gb_model = GradientBoostingRegressor(n_estimators=100, random_state=42)
gb_model.fit(X_train_scaled, y_train_scaled.ravel())
gb_predictions_scaled = gb_model.predict(X_test_scaled)
gb_predictions = scaler_y.inverse_transform(gb_predictions_scaled.reshape(-1, 1))

# Calculate evaluation metrics for Gradient Boosting
gb_mae = mean_absolute_error(y_test, gb_predictions)
gb_mse = mean_squared_error(y_test, gb_predictions)
gb_rmse = np.sqrt(gb_mse)

# Calculate percentage metrics for Gradient Boosting
gb_mae_percent = (gb_mae / mean_y_test) * 100
gb_rmse_percent = (gb_rmse / mean_y_test) * 100
gb_mse_percent = (gb_mse / (mean_y_test ** 2)) * 100

print(f"Gradient Boosting - Mean Absolute Error (MAE): {gb_mae} ({gb_mae_percent:.2f}%)")
print(f"Gradient Boosting - Mean Squared Error (MSE): {gb_mse} ({gb_mse_percent:.2f}%)")
print(f"Gradient Boosting - Root Mean Squared Error (RMSE): {gb_rmse} ({gb_rmse_percent:.2f}%)")

# Add GB predictions to the results dataframe
results_df['Predicted_GB'] = np.nan
results_df.loc[X_test.index, 'Predicted_GB'] = gb_predictions

plt.figure(figsize=(10, 6))
plt.plot(results_df['TIME_PERIOD'], y, label='Actual Values', color='b', linewidth = 0.8)
plt.plot(results_df['TIME_PERIOD'], results_df['Predicted_GB'], label='GB Predicted Values', color = 'r', linestyle = '--', marker = 'o', markersize = 1)
plt.xlabel('Time')
plt.ylabel(dependent_var)
plt.title(f'Actual vs Gradient Boosting Predicted Values for {dependent_var}')
plt.legend()
plt.grid(True)
plt.show()

importances_gb = gb_model.feature_importances_
indices_gb = np.argsort(importances_gb)[::-1]
features_gb = [independent_vars[i] for i in indices_gb]

# Print feature importances for Gradient Boosting
print("\nFeature Importances based on Gini Index (Gradient Boosting):")
for feature_name, importance in zip(features_gb, importances_gb[indices_gb]):
    print(f"{feature_name}: {importance:.4f}")

plt.figure(figsize=(10, 6))
plt.bar(features_gb, importances_gb[indices_gb], color='blue')
plt.axhline(y=0.05, color='red', linestyle='--', label='Significance Level (0.05)')
plt.xlabel('Variables')
plt.ylabel('Importance')
plt.title('Variable Importance Based on Gradient Boosting')
plt.xticks(rotation=45)
plt.legend()  # Add legend to show the significance level
plt.grid(True)
plt.show()

#-----------------------------------

# Define the scoring metrics to use in cross-validation
scoring = {'MAE': make_scorer(mean_absolute_error),
           'MSE': make_scorer(mean_squared_error),
           'RMSE': make_scorer(lambda y_true, y_pred: np.sqrt(mean_squared_error(y_true, y_pred)))}

# Linear Regression Cross-Validation (10-fold)
print("\nLinear Regression Cross-Validation (10-fold):")
cv_scores_lr = cross_val_score(model_lr, X_scaled, y, cv=10, scoring='neg_mean_squared_error')
cv_mae_lr = cross_val_score(model_lr, X_scaled, y, cv=10, scoring='neg_mean_absolute_error')

# Convert negative scores to positive for interpretation
cv_mse_lr = -cv_scores_lr
cv_mae_lr = -cv_mae_lr
cv_rmse_lr = np.sqrt(cv_mse_lr)

# Calculate average and standard deviation for the metrics
print(f"Linear Regression - Average MSE: {cv_mse_lr.mean():.4f} ± {cv_mse_lr.std():.4f}")
print(f"Linear Regression - Average MAE: {cv_mae_lr.mean():.4f} ± {cv_mae_lr.std():.4f}")
print(f"Linear Regression - Average RMSE: {cv_rmse_lr.mean():.4f} ± {cv_rmse_lr.std():.4f}")

# Random Forest Cross-Validation (10-fold)
print("\nRandom Forest Cross-Validation (10-fold):")
cv_scores_rf = cross_val_score(rf_model, X_scaled, y, cv=10, scoring='neg_mean_squared_error')
cv_mae_rf = cross_val_score(rf_model, X_scaled, y, cv=10, scoring='neg_mean_absolute_error')

# Convert negative scores to positive for interpretation
cv_mse_rf = -cv_scores_rf
cv_mae_rf = -cv_mae_rf
cv_rmse_rf = np.sqrt(cv_mse_rf)

# Calculate average and standard deviation for the metrics
print(f"Random Forest - Average MSE: {cv_mse_rf.mean():.4f} ± {cv_mse_rf.std():.4f}")
print(f"Random Forest - Average MAE: {cv_mae_rf.mean():.4f} ± {cv_mae_rf.std():.4f}")
print(f"Random Forest - Average RMSE: {cv_rmse_rf.mean():.4f} ± {cv_rmse_rf.std():.4f}")

# Gradient Boosting Cross-Validation (10-fold)
print("\nGradient Boosting Cross-Validation (10-fold):")
cv_scores_gb = cross_val_score(gb_model, X_scaled, y, cv=10, scoring='neg_mean_squared_error')
cv_mae_gb = cross_val_score(gb_model, X_scaled, y, cv=10, scoring='neg_mean_absolute_error')

# Convert negative scores to positive for interpretation
cv_mse_gb = -cv_scores_gb
cv_mae_gb = -cv_mae_gb
cv_rmse_gb = np.sqrt(cv_mse_gb)

# Calculate average and standard deviation for the metrics
print(f"Gradient Boosting - Average MSE: {cv_mse_gb.mean():.4f} ± {cv_mse_gb.std():.4f}")
print(f"Gradient Boosting - Average MAE: {cv_mae_gb.mean():.4f} ± {cv_mae_gb.std():.4f}")
print(f"Gradient Boosting - Average RMSE: {cv_rmse_gb.mean():.4f} ± {cv_rmse_gb.std():.4f}")


# MACHINE LEARNING

# ***Classification Analysis***

In [ ]:
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.ensemble import RandomForestClassifier
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score

# Read the dataset
dataset_file_path = f'EDA_ML_DATA/{country}/updated_merged_data_{country}.csv'
df = pd.read_csv(dataset_file_path)

# Convert the TIME_PERIOD column to datetime
df['TIME_PERIOD'] = pd.to_datetime(df['TIME_PERIOD'])

# Define dependent and independent variables for Team-1
#dependent_var = f'UN_{country}_T_TOT'  # Replace with your dependent variable
#independent_vars = ['Covid19_Trend_Binary', 'ECB_EXCHANGE_RATES_USD_EUR', f'Covid_{country}_Cumulative_Deaths', 'Oil_Close_Price', 'Corn_Close_Price']  # Replace with your independent variables

# Define dependent and independent variables for Team-3
dependent_var = f'INFLATION_{country}_VALUE'  # Replace with your dependent variable
independent_vars = ['Russia_Ukraine_War_VideoCount_Binary', 'Oil_Close_Price', f'Covid_{country}_Cumulative_Deaths', 'ECB_EXCHANGE_RATES_CNY_EUR', f'BS_{country}_FS_NY', f'UN_{country}_T_TOT', 'Covid19_Trend_Binary', 'ECB_EXCHANGE_RATES_USD_EUR']  # Replace with your independent variables

# Create X and y
X = df[independent_vars]
y = df[dependent_var]

# Z-score normalization
scaler_X = StandardScaler()
X_scaled = scaler_X.fit_transform(X)

# Check if the target variable is already binary (0 and 1)
if set(y.unique()) == {0, 1}:
    y_class = y
else:
    # Create a binary classification for the dependent variable
    median_value = y.median()
    y_class = (y > median_value).astype(int)  # 1 if the value is above the median, otherwise 0
    print(f"Class 1: > {median_value}, Class 0: ≤ {median_value}")

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y_class, test_size=0.2, random_state=42)

# Logistic Regression Model
model_lr = LogisticRegression(random_state=42)
model_lr.fit(X_train, y_train)

# Predict the classes with Logistic Regression
y_pred_class_lr = model_lr.predict(X_test)

# Evaluate Logistic Regression
accuracy_lr = accuracy_score(y_test, y_pred_class_lr)
print(f'Logistic Regression Classification Accuracy: {accuracy_lr}')
print('Logistic Regression Classification Report:')
print(classification_report(y_test, y_pred_class_lr))
print('Logistic Regression Confusion Matrix:')
print(confusion_matrix(y_test, y_pred_class_lr))

# Visualize the results of Logistic Regression
plt.figure(figsize=(10, 6))
plt.scatter(df.iloc[y_test.index]['TIME_PERIOD'], y_test, label='Actual Class', color='b', alpha=0.6)
plt.scatter(df.iloc[y_test.index]['TIME_PERIOD'], y_pred_class_lr, label='Predicted Class', color='r', alpha=0.6, marker='x')
plt.xlabel('Time')
plt.ylabel('Class')
if 'median_value' in locals():
    plt.title(f'Actual vs Predicted Classes for {dependent_var} - Logistic Regression\nClass 1: > {median_value}, Class 0: ≤ {median_value}')
else:
    plt.title(f'Actual vs Predicted Classes for {dependent_var} - Logistic Regression')
plt.legend()
plt.grid(True)
plt.show()

# Feature importance based on Logistic Regression coefficients
coefficients = model_lr.coef_[0]
indices = np.argsort(abs(coefficients))[::-1]

# Print the feature ranking based on the absolute value of coefficients
print("Feature ranking based on Logistic Regression coefficients (absolute value):")

for f in range(X.shape[1]):
    print(f"{f + 1}. feature {independent_vars[indices[f]]} (coefficient: {coefficients[indices[f]]})")

# Plot the feature importances based on Logistic Regression coefficients
plt.figure(figsize=(10, 6))
plt.title("Feature Importance Based on Logistic Regression Coefficients")
plt.bar(range(X.shape[1]), coefficients[indices], align="center")
plt.xticks(range(X.shape[1]), [independent_vars[i] for i in indices], rotation=90)
plt.xlim([-1, X.shape[1]])
plt.ylabel('Coefficient Value')
plt.tight_layout()
plt.grid(True)
plt.show()

# Random Forest Model
model_rf = RandomForestClassifier(n_estimators=100, random_state=42)
model_rf.fit(X_train, y_train)

# Predict the classes with Random Forest
y_pred_class_rf = model_rf.predict(X_test)

# Evaluate Random Forest
accuracy_rf = accuracy_score(y_test, y_pred_class_rf)
print(f'Random Forest Classification Accuracy: {accuracy_rf}')
print('Random Forest Classification Report:')
print(classification_report(y_test, y_pred_class_rf))
print('Random Forest Confusion Matrix:')
print(confusion_matrix(y_test, y_pred_class_rf))

# Visualize the results of Random Forest
plt.figure(figsize=(10, 6))
plt.scatter(df.iloc[y_test.index]['TIME_PERIOD'], y_test, label='Actual Class', color='b', alpha=0.6)
plt.scatter(df.iloc[y_test.index]['TIME_PERIOD'], y_pred_class_rf, label='Predicted Class', color='r', alpha=0.6, marker='x')
plt.xlabel('Time')
plt.ylabel('Class')
if 'median_value' in locals():
    plt.title(f'Actual vs Predicted Classes for {dependent_var} - Random Forest Classification\nClass 1: > {median_value}, Class 0: ≤ {median_value}')
else:
    plt.title(f'Actual vs Predicted Classes for {dependent_var} - Random Forest Classification')
plt.legend()
plt.grid(True)
plt.show()

# Feature importance based on Gini index from Random Forest
importances = model_rf.feature_importances_
indices = np.argsort(importances)[::-1]

# Print the feature ranking
print("Feature ranking based on Gini index:")

for f in range(X.shape[1]):
    print(f"{f + 1}. feature {independent_vars[indices[f]]} ({importances[indices[f]]})")

# Plot the feature importances of the forest
plt.figure(figsize=(10, 6))
plt.title("Feature Importance Based on Gini Index (Random Forest)")
plt.bar(range(X.shape[1]), importances[indices], align="center")
plt.xticks(range(X.shape[1]), [independent_vars[i] for i in indices], rotation=90)
plt.xlim([-1, X.shape[1]])
plt.ylabel('Importance')
plt.tight_layout()
plt.grid(True)
plt.show()

#-----------------------------

# Logistic Regression Model with Cross-Validation
model_lr_cv = LogisticRegression(random_state=42)
cv_scores_lr = cross_val_score(model_lr_cv, X_scaled, y_class, cv=10)  # 10-fold cross-validation

# Print Logistic Regression Cross-Validation Results
print(f'Logistic Regression Cross-Validation Accuracy: {cv_scores_lr.mean()} ± {cv_scores_lr.std()}')

# Random Forest Model with Cross-Validation
model_rf_cv = RandomForestClassifier(n_estimators=100, random_state=42)
cv_scores_rf = cross_val_score(model_rf_cv, X_scaled, y_class, cv=10)  # 10-fold cross-validation

# Print Random Forest Cross-Validation Results
print(f'Random Forest Cross-Validation Accuracy: {cv_scores_rf.mean()} ± {cv_scores_rf.std()}')

# Train Logistic Regression on the entire dataset for feature importance
model_lr_cv.fit(X_scaled, y_class)

# Feature importance based on Logistic Regression coefficients
coefficients_cv = model_lr_cv.coef_[0]
indices_lr_cv = np.argsort(abs(coefficients_cv))[::-1]

# Print the feature ranking based on Logistic Regression coefficients with cross-validation
print("Feature ranking based on Logistic Regression coefficients (absolute value) with cross-validation:")
for f in range(X.shape[1]):
    print(f"{f + 1}. feature {independent_vars[indices_lr_cv[f]]} (coefficient: {coefficients_cv[indices_lr_cv[f]]})")

# Train Random Forest on the entire dataset for feature importance
model_rf_cv.fit(X_scaled, y_class)

# Feature importance based on Gini index from Random Forest with cross-validation
importances_rf_cv = model_rf_cv.feature_importances_
indices_rf_cv = np.argsort(importances_rf_cv)[::-1]

# Print the feature ranking based on Gini index with cross-validation
print("Feature ranking based on Gini index (Random Forest) with cross-validation:")
for f in range(X.shape[1]):
    print(f"{f + 1}. feature {independent_vars[indices_rf_cv[f]]} ({importances_rf_cv[indices_rf_cv[f]]})")


# ***Clustering Analysis***

In [ ]:
import pandas as pd
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.cluster import KMeans, AgglomerativeClustering
import matplotlib.pyplot as plt
from sklearn.metrics import silhouette_score
from scipy.cluster.hierarchy import dendrogram, linkage

# Read the dataset
dataset_file_path = f'EDA_ML_DATA/{country}/updated_merged_data_{country}.csv'
df = pd.read_csv(dataset_file_path)

# Define the new sets of features
economic_unemployment_features = [
    'ESTR_EU000A2QQF08_CI', f'CISS_{country}_SS_CIN', f'GOVDEBT_{country}', #'ESTR_EU000A2X2A25_TT',
    f'INFLATION_{country}_VALUE', f'GDP_GROWTH_{country}',  
    f'UN_{country}_T_TOT'
]

market_risk_and_confidence_features = [
    f'BS_{country}_CSMCI', f'BS_{country}_FS_NY', f'BS_{country}_GES_NY', f'BS_{country}_MP_NY', 
    f'BS_{country}_PT_NY', f'BS_{country}_SV_NY', 'ECB_EXCHANGE_RATES_USD_EUR',
    'ECB_EXCHANGE_RATES_CNY_EUR', 'Oil_Close_Price', 'Gas_Close_Price'    
]

# Function to perform clustering and visualize results
def perform_clustering(features, feature_set_name, n_clusters):
    # Create a new DataFrame with only the selected features
    df_features = df[features]

    # Normalize the data with StandardScaler
    scaler = StandardScaler()
    df_features_scaled = scaler.fit_transform(df_features)
    
    # Elbow Method and Silhouette Scores for determining the optimal number of clusters
    inertia = []
    silhouette_scores = []
    K = range(2, 11)

    for k in K:
        kmeans = KMeans(n_clusters=k, random_state=42)
        kmeans.fit(df_features_scaled)
        inertia.append(kmeans.inertia_)
        silhouette_scores.append(silhouette_score(df_features_scaled, kmeans.labels_))

    # Plot the Elbow Method
    plt.figure(figsize=(10, 6))
    plt.plot(K, inertia, 'bo-')
    plt.xlabel('Number of clusters (k)')
    plt.ylabel('Inertia')
    plt.title(f'Elbow Method for Optimal Number of Clusters ({feature_set_name})')
    plt.grid(True)
    plt.show()

    # Plot the Silhouette Scores
    plt.figure(figsize=(10, 6))
    plt.plot(K, silhouette_scores, 'bo-')
    plt.xlabel('Number of clusters (k)')
    plt.ylabel('Silhouette Score')
    plt.title(f'Silhouette Scores for Optimal Number of Clusters ({feature_set_name})')
    plt.grid(True)
    plt.show()

    # Apply K-means clustering with the specified number of clusters
    kmeans = KMeans(n_clusters=n_clusters, random_state=42)
    df[f'KMeans_Cluster_{feature_set_name}'] = kmeans.fit_predict(df_features_scaled)

    # Visualize the K-means clustering results using PCA
    from sklearn.decomposition import PCA

    pca = PCA(n_components=2)
    df_pca = pca.fit_transform(df_features_scaled)

    plt.figure(figsize=(10, 6))
    plt.scatter(df_pca[:, 0], df_pca[:, 1], c=df[f'KMeans_Cluster_{feature_set_name}'], cmap='viridis')
    plt.xlabel('Principal Component 1')
    plt.ylabel('Principal Component 2')
    plt.title(f'K-means Clustering of {feature_set_name}')
    plt.colorbar(label='Cluster')
    plt.grid(True)
    plt.show()

    # Analyze clusters by calculating mean values of the features within each cluster
    kmeans_cluster_means = df.groupby(f'KMeans_Cluster_{feature_set_name}')[features].mean()

    # Print mean values for interpretation
    print(f"\nMean values of {feature_set_name} for each K-means cluster:")
    print(kmeans_cluster_means)

    # Visualize the mean values to help with interpretation
    #fig, ax = plt.subplots(figsize=(16, 10))  # Adjusting the figsize to make the plot larger
    #kmeans_cluster_means.plot(kind='bar', ax=ax, title=f'Mean Values of {feature_set_name} by K-means Cluster', logy=True)
    #ax.legend(bbox_to_anchor=(1.05, 1), loc='upper left')  # Move the legend outside of the plot
    #plt.tight_layout()
    #plt.show()
    
    # Visualize the mean values to help with interpretation
    fig, ax = plt.subplots(figsize=(16, 10))  # Adjusting the figsize to make the plot larger
    kmeans_cluster_means.plot(kind='bar', ax=ax, title=f'Mean Values of {feature_set_name} by K-means Cluster')
    
    # Προσαρμογή του εύρους του άξονα y για να περιλαμβάνει αρνητικές και θετικές τιμές
    ax.set_ylim([kmeans_cluster_means.values.min() - 1, kmeans_cluster_means.values.max() + 1])
    
    ax.legend(bbox_to_anchor=(1.05, 1), loc='upper left')  # Move the legend outside of the plot
    plt.tight_layout()
    plt.show()


# Define the number of clusters for each feature set
economic_unemployment_clusters = 4  # Change this value to the desired number of clusters for economic and unemployment features
market_risk_confidence_clusters = 3  # Change this value to the desired number of clusters for market risk and confidence features

# Perform clustering on economic and unemployment features
perform_clustering(economic_unemployment_features, 'Economic and Unemployment Features', economic_unemployment_clusters)

# Perform clustering on market risk features
perform_clustering(market_risk_and_confidence_features, 'Market Risk and Confidence Features', market_risk_confidence_clusters)


#---------------------------

# Function to perform Agglomerative Clustering and visualize results
def perform_agglomerative_clustering(features, feature_set_name, n_clusters):
    # Create a new DataFrame with only the selected features
    df_features = df[features]

    # Normalize the data with StandardScaler
    scaler = StandardScaler()
    df_features_scaled = scaler.fit_transform(df_features)
    
    # Normalize the data with Min-Max Scaler
    #scaler = MinMaxScaler()
    #df_features_scaled = scaler.fit_transform(df_features)

    # Apply Agglomerative Clustering
    agglomerative = AgglomerativeClustering(n_clusters=n_clusters)
    df[f'Agglomerative_Cluster_{feature_set_name}'] = agglomerative.fit_predict(df_features_scaled)

    # Visualize the Agglomerative Clustering results using PCA
    from sklearn.decomposition import PCA

    pca = PCA(n_components=2)
    df_pca = pca.fit_transform(df_features_scaled)

    plt.figure(figsize=(10, 6))
    plt.scatter(df_pca[:, 0], df_pca[:, 1], c=df[f'Agglomerative_Cluster_{feature_set_name}'], cmap='viridis')
    plt.xlabel('Principal Component 1')
    plt.ylabel('Principal Component 2')
    plt.title(f'Agglomerative Clustering of {feature_set_name}')
    plt.colorbar(label='Cluster')
    plt.grid(True)
    plt.show()

    # Generate and plot the dendrogram
    Z = linkage(df_features_scaled, 'ward')
    plt.figure(figsize=(10, 6))
    dendrogram(Z)
    plt.title(f'Dendrogram for {feature_set_name}')
    plt.xlabel('Samples')
    plt.ylabel('Distance')
    plt.show()

    # Analyze clusters by calculating mean values of the features within each cluster
    agglomerative_cluster_means = df.groupby(f'Agglomerative_Cluster_{feature_set_name}')[features].mean()

    # Print mean values for interpretation
    print(f"\nMean values of {feature_set_name} for each Agglomerative cluster:")
    print(agglomerative_cluster_means)

    # Visualize the mean values to help with interpretation
    #fig, ax = plt.subplots(figsize=(16, 10))  # Adjusting the figsize to make the plot larger
    #agglomerative_cluster_means.plot(kind='bar', ax=ax, title=f'Mean Values of {feature_set_name} by Agglomerative Cluster', logy=True)
    #ax.legend(bbox_to_anchor=(1.05, 1), loc='upper left')  # Move the legend outside of the plot
    #plt.tight_layout()
    #plt.show()
    
    # Visualize the mean values to help with interpretation
    fig, ax = plt.subplots(figsize=(16, 10))  # Adjusting the figsize to make the plot larger
    agglomerative_cluster_means.plot(kind='bar', ax=ax, title=f'Mean Values of {feature_set_name} by Agglomerative Cluster')
    
    ax.set_ylim([agglomerative_cluster_means.values.min() - 1, agglomerative_cluster_means.values.max() + 1])
    
    ax.legend(bbox_to_anchor=(1.05, 1), loc='upper left')  # Move the legend outside of the plot
    plt.tight_layout()
    plt.show()
    
    # Define the number of clusters for each feature set
economic_unemployment_clusters = 4  # Change this value to the desired number of clusters for economic and unemployment features
market_risk_confidence_clusters = 3  # Change this value to the desired number of clusters for market risk and confidence features

# Perform Agglomerative Clustering on economic and unemployment features
perform_agglomerative_clustering(economic_unemployment_features, 'Economic and Unemployment Features', economic_unemployment_clusters)

# Perform Agglomerative Clustering on market risk features
perform_agglomerative_clustering(market_risk_and_confidence_features, 'Market Risk and Confidence Features', market_risk_confidence_clusters)
